# 플라스크 심플 웹서버 #

In [14]:
'''
CREATE TABLE coronafact (
  num int(11) NOT NULL AUTO_INCREMENT,
  udate varchar(255) NOT NULL,
  confirmator varchar(100) NOT NULL,
  dead varchar(50) NOT NULL,
  suspected varchar(50) NOT NULL,
  inspection varchar(50) NOT NULL,
  rdate datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;
END'''

'\nCREATE TABLE coronafact (\n  num int(11) NOT NULL AUTO_INCREMENT,\n  udate varchar(255) NOT NULL,\n  confirmator varchar(100) NOT NULL,\n  dead varchar(50) NOT NULL,\n  suspected varchar(50) NOT NULL,\n  inspection varchar(50) NOT NULL,\n  rdate datetime DEFAULT NOW(),\n  primary key (num)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;\nEND'

In [15]:
%mkdir templates -p

In [16]:
%%writefile templates/index.html
<!DOCTYPE html>
<html xml:lang='ko' xmlns='http://www.w3.org/1999/xhtml'>
  <head>
    <meta http-equiv='Content-Type' content='text/html' charset='utf-8'/>
    <meta name='viewport' content='user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, width=device-width'/>
    <title>부산IT융합부품연구소</title>
  </head>
  <body>
    <table>
      <tr>
        <td><h2><a href="https://github.com/gyunseul9" target="_blank">Github</a></h2></td>
        <td><h2><a href="https://blog.naver.com/napple" target="_blank">Naver Blog</a></h2></td>
      </tr>
      <tr>
        <td><h2><a href="https://lonbehold.tistory.com/" target="_blank">Tistory</a></h2></td>
        <td><h2><a href="https://lonbekim.blogspot.com/" target="_blank">Google Blogspot</a></h2></td>
      </tr>        
	   	<tr>
        <td colspan="2"><a href="/"><a href="/list?keyword=coronafact&limit=10"><h3>코로나 실시간 상황</h3></a></td>
      </tr> 	    	 	             	         
    </table>
  </body>
</html>

Overwriting templates/index.html


In [17]:
%%writefile templates/coronafact.html
<!DOCTYPE html>
<html xml:lang='ko' xmlns='http://www.w3.org/1999/xhtml'>
	<head>
		<meta http-equiv='Content-Type' content='text/html' charset='utf-8'/>
		<meta name='viewport' content='user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, width=device-width'/>
		<title>코로나 실시간 상황</title>  
	</head>
    <body>
        <table border="1">
        <tr><td><a href="/">top</a></td>
        </tr>
            {% for item in data %}
            <tr>
                <td>
                    출처: <a href="http://coronafact.org" target="_blank">KMA 코로나 팩트</a><br>
                    업데이트: {{item['udate']}}<br>
                    확진자: {{item['confirmator']}}<br>
                    사망자: {{item['dead']}}<br>
                    의심환자: {{item['suspected']}}<br>
                    검사중: {{item['inspection']}}<br>
        </td>
            </tr>
        {% endfor %}
    </table>
    </body>

Overwriting templates/coronafact.html


In [18]:
!sudo pip3 install pymysql

In [19]:
!sudo pip3 install flask-ngrok

In [31]:
import pymysql
import configparser
from datetime import datetime
from google.colab import drive
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

In [32]:
c=drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
CNF_INI = 'drive/My Drive/development/config/openapi.ini'
ERR_URI = 'drive/My Drive/development/test/error.log'

In [34]:
config = configparser.ConfigParser()
config.read(CNF_INI)
HOST = config['mysql']['host']
USER = config['mysql']['user']
PASSWORD = config['mysql']['password']
DATABASE = config['mysql']['database']
PORT = config['mysql']['port']
CHARSET = config['mysql']['charset']

In [35]:
def db_access():
  conn = pymysql.connect(
      host=HOST,
      user=USER,
      password=PASSWORD,
      db=DATABASE,
      charset=CHARSET,
      port=int(PORT),
      cursorclass=pymysql.cursors.DictCursor)
  return conn

In [36]:
def exec_list(conn,keyword,limit):
  with conn.cursor() as cursor:
    query = get_only_list(keyword,limit)
    cursor.execute(query)
    data = cursor.fetchall()
    return data   

In [37]:
def get_only_list(keyword,limit):
  query = 'select * from {} order by num desc limit {}'.format(keyword,limit)
  print(query)
  return query 

In [40]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def index():
  return render_template('index.html')

@app.route('/list')
def list():
  keyword = request.args.get('keyword', default = 'coronafact', type = str)
  limit = request.args.get('limit', default = '10', type = str)
    
  conn = db_access()
    
  data = exec_list(conn,keyword,limit)

  session = {}

  session['keyword'] = str(keyword)
  session['limit'] = str(limit)
  
  return render_template('/coronafact.html',data=data,session=session)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://014ac8e4b57c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Aug/2020 07:47:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Aug/2020 07:47:32] "GET /list?keyword=coronafact&limit=10 HTTP/1.1" 200 -


select * from coronafact order by num desc limit 10


127.0.0.1 - - [18/Aug/2020 07:47:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 07:47:37] "GET /list?keyword=coronafact&limit=10 HTTP/1.1" 200 -


select * from coronafact order by num desc limit 10
